# Stack frame

See also: https://realpython.com/python-scope-legb-rule/

## Get current function name

In [1]:
import inspect
import sys

def print_my_name():
    """Print this function name using various methods.
    
    Methods sorted from fastest:
        * sys._getframe() - fast but uses a private name, accepts frame hierarchy number
        * inspect.currentframe() - can access higher frames only through .f_back
        * inspect.stack() - about 2000x slower than inspect.currentframe()
    
    Origin:
        https://stackoverflow.com/a/17366561/320437
    """
    print(inspect.currentframe().f_code.co_name)
    print(inspect.stack()[0].function)

    print(inspect.stack()[0].frame.f_code.co_name)
    
    print(inspect.stack()[0][0].f_code.co_name) # .frame možno vybrat indexem
    print(inspect.stack()[0][3])                # .function možno vybrat indexem

    # sys._getframe() může být rychlejší
    print(sys._getframe().f_code.co_name)
    print(sys._getframe(0).f_code.co_name)      # volitelně index pro vyšší frame!


print_my_name()

print_my_name
print_my_name
print_my_name
print_my_name
print_my_name
print_my_name
print_my_name


### Function to get current scope name

In [2]:
import inspect

def scope_name(frames_up=0) -> str:
    """Get name of the scope this function is executed from."""
    return inspect.stack()[1 + frames_up].function

def print_scope_name(description='', frames_up=0):
    description = f'Scope name{f" in {description}" if description else ""}:'
    print(f'{description:35} {scope_name(1 + frames_up)}')

In [3]:
print_scope_name('notebook cell')

print_scope_name('above notebook cell', frames_up=1)

print_scope_name('top level', frames_up=-3)   # -3 translates to -1 inside scope_name()

class ExampleClass:
    print_scope_name('class')

[
    print_scope_name('list comprehension')
    for _ in (None,)
]

(lambda: print_scope_name('lambda'))()

# There is no new scope for function parameters
def dummy_function(_=print_scope_name('function parameters')):
    pass

def example_function():
    print_scope_name('function')

example_function()

Scope name in notebook cell:        <cell line: 1>
Scope name in above notebook cell:  run_code
Scope name in top level:            _run_module_as_main
Scope name in class:                ExampleClass
Scope name in list comprehension:   <listcomp>
Scope name in lambda:               <lambda>
Scope name in function parameters:  <cell line: 18>
Scope name in function:             example_function


### Decorator to give function's name to the function as an argument

In [4]:
import functools

def named(original_function):
    """Define decorator giving the wrapped function its name as the first argument.
    
    Origin:
        https://stackoverflow.com/a/61788034/320437
    """
    @functools.wraps(original_function)
    def wrapper(*args, **kwargs):
        return original_function(original_function.__name__, *args, **kwargs)
    return wrapper

In [5]:
@named
def my_func(name: str, argument: str) -> None:
    print(f'function name: {name}')
    print(f'{argument = }')

my_func('hello, world')

function name: my_func
argument = 'hello, world'
